# A3 Scaling Law Pilot Experiment: GPT-3.5-turbo

**Purpose**: Test whether λ* (Backfire boundary) correlates with model capability

**Hypothesis**: If negative correlation exists (smarter = more susceptible),
GPT-3.5 (lower capability) should have HIGHER λ* than GPT-4o and Claude.

**Current Data (2 points)**:
| Model | Baseline | λ* |
|-------|----------|----|
| GPT-4o | 56.28% | 0.865 |
| Claude | 75.88% | 0.449 |

**Expected Result**:
- GPT-3.5 Baseline: ~30-50% (lower than both)
- If negative correlation: GPT-3.5 λ* > 0.865
- If positive correlation: GPT-3.5 λ* < 0.449

**This is a PILOT (50 problems) to check feasibility before full A3 experiment.**

## 0. Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXPERIMENT_VERSION = 'a3_pilot_gpt35'
EXPERIMENT_DATE = datetime.now().strftime('%Y%m%d')

SAVE_DIR = '/content/drive/MyDrive/CoT_Experiment'
SAVE_DIR_EXP = f'{SAVE_DIR}/a3_scaling_pilot_{EXPERIMENT_DATE}'

os.makedirs(SAVE_DIR_EXP, exist_ok=True)
os.makedirs(f'{SAVE_DIR_EXP}/results', exist_ok=True)
os.makedirs(f'{SAVE_DIR_EXP}/checkpoints', exist_ok=True)

print(f'A3 Pilot experiment save directory: {SAVE_DIR_EXP}')

## 1. Install Dependencies

In [ ]:
!pip install datasets openai pandas tqdm matplotlib scipy -q
print('Dependencies installed.')

## 2. Configuration

**CRITICAL**: Use same GLOBAL_SEED and problem selection as original experiments

In [ ]:
import json
import re
import random
import time
import hashlib
from typing import List, Dict, Optional, Any
from dataclasses import dataclass, asdict
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np

# =============================================================================
# Configuration - MUST MATCH ORIGINAL EXPERIMENTS
# =============================================================================
GLOBAL_SEED = 20251224  # Same as Claude/GPT-4o experiments

# PILOT: Reduced problem set for quick feasibility check
N_PROBLEMS_PILOT = 50  # Reduced from 200 for pilot

# Experimental conditions
I_FIXED = 10  # Same trace depth
LAMBDA_VALUES = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]  # Same as original

# =============================================================================
# MODEL: GPT-3.5-turbo (lower capability baseline)
# =============================================================================
API_MODEL = 'gpt-3.5-turbo'
API_MAX_TOKENS = 256
API_RATE_LIMIT_DELAY = 0.2  # GPT-3.5 is fast
CHECKPOINT_EVERY = 25

print('='*70)
print('A3 SCALING LAW PILOT: GPT-3.5-turbo')
print('='*70)
print(f'  GLOBAL_SEED: {GLOBAL_SEED}')
print(f'  N_PROBLEMS (pilot): {N_PROBLEMS_PILOT}')
print(f'  I (fixed): {I_FIXED}')
print(f'  λ values: {LAMBDA_VALUES}')
print(f'  Model: {API_MODEL}')
print(f'  Total inferences: {N_PROBLEMS_PILOT * (len(LAMBDA_VALUES) + 1)}')
print('='*70)
print('\n⚠️  This is a PILOT experiment to check A3 feasibility')
print('    If results are promising, run full 200-problem experiment')

## 3. Data Structures & Utilities

In [ ]:
@dataclass
class GSM8KProblem:
    index: int
    question: str
    answer_text: str
    final_answer: int

@dataclass
class ExperimentResult:
    problem_index: int
    condition: str
    I: Optional[int]
    lam: Optional[float]
    A_target: Optional[float]
    model_answer: Optional[int]
    correct_answer: int
    is_correct: bool
    raw_output: str
    timestamp: str
    model: str

def extract_final_answer(answer_text: str) -> int:
    match = re.search(r'####\s*([\d,]+)', answer_text)
    if match:
        return int(match.group(1).replace(',', ''))
    raise ValueError(f'Could not extract final answer')

def save_json(data: Any, filepath: str):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f'Saved: {filepath}')

def load_json(filepath: str) -> Any:
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def derive_seed(global_seed: int, problem_id: int, I: int, lam: float) -> int:
    key = f"{global_seed}|{problem_id}|I={I}|lam={lam}"
    h = hashlib.sha256(key.encode("utf-8")).hexdigest()
    return int(h[:8], 16)

## 4. Load GSM8K and Select Problems

**IMPORTANT**: Use same seed to select from the SAME 200 problems as original,
then take first 50 for pilot.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('gsm8k', 'main', split='test')
print(f'GSM8K test set loaded: {len(dataset)} problems')

def select_problems(dataset, n_problems: int, seed: int) -> List[int]:
    rng = random.Random(seed)
    indices = list(range(len(dataset)))
    rng.shuffle(indices)
    return sorted(indices[:n_problems])

# Select same 200 as original, then take first N for pilot
all_200_indices = select_problems(dataset, 200, GLOBAL_SEED)
selected_indices = all_200_indices[:N_PROBLEMS_PILOT]

print(f'Selected {len(selected_indices)} problems for pilot')
print(f'(Subset of original 200 problems)')

problems = []
for idx in selected_indices:
    item = dataset[idx]
    try:
        final_ans = extract_final_answer(item['answer'])
        prob = GSM8KProblem(
            index=idx,
            question=item['question'],
            answer_text=item['answer'],
            final_answer=final_ans
        )
        problems.append(prob)
    except ValueError as e:
        print(f'Skipping problem {idx}: {e}')

print(f'\nLoaded {len(problems)} problems for experiment')

## 5. OpenAI API Setup

In [ ]:
from google.colab import userdata
from openai import OpenAI

# Get API key from Colab secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

def call_openai(prompt: str, max_tokens: int = API_MAX_TOKENS) -> str:
    """Call OpenAI API with retry logic"""
    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model=API_MODEL,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=max_tokens,
                temperature=0  # Deterministic
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f'API error (attempt {attempt+1}): {e}')
            time.sleep(2 ** attempt)
    return ""

# Test API
test_response = call_openai("What is 2+2? Reply with just the number.")
print(f'API test: {test_response}')
print(f'Model: {API_MODEL}')

## 6. CoT Generation Functions

In [ ]:
def generate_clean_cot(problem: GSM8KProblem, I: int) -> List[str]:
    """
    Generate I clean reasoning steps from GSM8K answer.
    Uses the ground truth answer text, parsed into steps.
    """
    # Parse answer text into steps
    lines = problem.answer_text.split('\n')
    steps = [l.strip() for l in lines if l.strip() and not l.strip().startswith('####')]
    
    # Pad or truncate to exactly I steps
    if len(steps) >= I:
        return steps[:I]
    else:
        # Pad with generic steps
        while len(steps) < I:
            steps.append(f"Step {len(steps)+1}: Continue calculation.")
        return steps

def generate_corrupted_step(problem: GSM8KProblem, step_idx: int, rng: random.Random) -> str:
    """
    Generate a corrupted reasoning step.
    Introduces plausible but incorrect calculations.
    """
    corruption_templates = [
        f"Step {step_idx+1}: Let's multiply the values: {rng.randint(10, 100)} × {rng.randint(2, 10)} = {rng.randint(100, 1000)}.",
        f"Step {step_idx+1}: Adding the totals: {rng.randint(50, 200)} + {rng.randint(50, 200)} = {rng.randint(100, 500)}.",
        f"Step {step_idx+1}: The difference is: {rng.randint(100, 500)} - {rng.randint(20, 100)} = {rng.randint(50, 400)}.",
        f"Step {step_idx+1}: Dividing gives us: {rng.randint(100, 1000)} ÷ {rng.randint(2, 10)} = {rng.randint(10, 200)}.",
        f"Step {step_idx+1}: Converting: {rng.randint(1, 10)} × {rng.randint(10, 100)} = {rng.randint(10, 1000)}."
    ]
    return rng.choice(corruption_templates)

def generate_mixed_cot(problem: GSM8KProblem, I: int, lam: float, seed: int) -> List[str]:
    """
    Generate CoT with λ proportion of corrupted steps.
    λ=0: all clean, λ=1: all corrupted
    """
    rng = random.Random(seed)
    clean_steps = generate_clean_cot(problem, I)
    
    n_corrupt = int(round(I * lam))
    corrupt_indices = set(rng.sample(range(I), n_corrupt)) if n_corrupt > 0 else set()
    
    mixed_steps = []
    for i in range(I):
        if i in corrupt_indices:
            mixed_steps.append(generate_corrupted_step(problem, i, rng))
        else:
            mixed_steps.append(clean_steps[i] if i < len(clean_steps) else f"Step {i+1}: Continue.")
    
    return mixed_steps

## 7. Experiment Execution Functions

In [ ]:
def create_direct_prompt(problem: GSM8KProblem) -> str:
    return f"""Solve this math problem. Give ONLY the final numerical answer in JSON format.

Problem: {problem.question}

Reply with ONLY: {{"final": <number>}}"""

def create_cot_prompt(problem: GSM8KProblem, cot_steps: List[str]) -> str:
    steps_text = '\n'.join(cot_steps)
    return f"""Here is a math problem with a provided reasoning trace.
Follow the reasoning and give the final numerical answer in JSON format.

Problem: {problem.question}

Reasoning trace:
{steps_text}

Based on this reasoning, what is the final answer?
Reply with ONLY: {{"final": <number>}}"""

def parse_answer(response: str) -> Optional[int]:
    """Extract numerical answer from model response"""
    # Try JSON format
    try:
        match = re.search(r'\{[^}]*"final"\s*:\s*(\d+)[^}]*\}', response)
        if match:
            return int(match.group(1))
    except:
        pass
    
    # Try plain number
    try:
        numbers = re.findall(r'\b(\d+)\b', response)
        if numbers:
            return int(numbers[-1])
    except:
        pass
    
    return None

## 8. Run Direct Condition (Baseline)

In [ ]:
print('='*60)
print('PHASE 1: Direct Condition (Baseline)')
print('='*60)

direct_results = []

for i, problem in enumerate(tqdm(problems, desc='Direct')):
    prompt = create_direct_prompt(problem)
    response = call_openai(prompt)
    
    answer = parse_answer(response)
    is_correct = (answer == problem.final_answer) if answer is not None else False
    
    result = {
        'problem_index': problem.index,
        'condition': 'direct',
        'model': API_MODEL,
        'model_answer': answer,
        'correct_answer': problem.final_answer,
        'is_correct': is_correct,
        'raw_output': response,
        'timestamp': datetime.now().isoformat()
    }
    direct_results.append(result)
    
    time.sleep(API_RATE_LIMIT_DELAY)
    
    if (i + 1) % CHECKPOINT_EVERY == 0:
        save_json(direct_results, f'{SAVE_DIR_EXP}/checkpoints/direct_checkpoint_{i+1}.json')

# Save final direct results
save_json(direct_results, f'{SAVE_DIR_EXP}/results/direct_results_gpt35.json')

# Calculate baseline accuracy
gpt35_direct_acc = sum(r['is_correct'] for r in direct_results) / len(direct_results)
print(f'\n✓ GPT-3.5 Direct (Baseline) Accuracy: {gpt35_direct_acc:.1%}')
print(f'  (Compare: GPT-4o = 56.28%, Claude = 75.88%)')

## 9. Run CoT Conditions (λ sweep)

In [ ]:
print('='*60)
print('PHASE 2: CoT Conditions (λ sweep)')
print('='*60)

cot_results = []
total_trials = len(problems) * len(LAMBDA_VALUES)

with tqdm(total=total_trials, desc='CoT sweep') as pbar:
    for lam in LAMBDA_VALUES:
        print(f'\n--- λ = {lam} ---')
        
        for problem in problems:
            # Generate mixed CoT
            seed = derive_seed(GLOBAL_SEED, problem.index, I_FIXED, lam)
            cot_steps = generate_mixed_cot(problem, I_FIXED, lam, seed)
            
            # Query model
            prompt = create_cot_prompt(problem, cot_steps)
            response = call_openai(prompt)
            
            answer = parse_answer(response)
            is_correct = (answer == problem.final_answer) if answer is not None else False
            
            result = {
                'problem_index': problem.index,
                'condition': 'cot',
                'model': API_MODEL,
                'I': I_FIXED,
                'lam': lam,
                'A_target': 1 - lam,
                'model_answer': answer,
                'correct_answer': problem.final_answer,
                'is_correct': is_correct,
                'raw_output': response,
                'timestamp': datetime.now().isoformat()
            }
            cot_results.append(result)
            
            time.sleep(API_RATE_LIMIT_DELAY)
            pbar.update(1)
        
        # Checkpoint after each λ
        save_json(cot_results, f'{SAVE_DIR_EXP}/checkpoints/cot_checkpoint_lam{lam}.json')

# Save final CoT results
save_json(cot_results, f'{SAVE_DIR_EXP}/results/cot_results_gpt35.json')
print('\n✓ CoT experiment complete!')

## 10. Analyze Results

In [ ]:
import matplotlib.pyplot as plt

# Convert to DataFrame
cot_df = pd.DataFrame(cot_results)

# Calculate accuracy by λ
gpt35_acc_by_lam = cot_df.groupby('lam')['is_correct'].mean().to_dict()

print('='*60)
print('GPT-3.5 RESULTS')
print('='*60)
print(f'\nDirect (Baseline): {gpt35_direct_acc:.1%}')
print('\nCoT accuracy by λ:')
for lam, acc in sorted(gpt35_acc_by_lam.items()):
    marker = '←' if acc < gpt35_direct_acc else ''
    print(f'  λ={lam:.1f}: {acc:.1%} {marker}')

## 11. Estimate λ* for GPT-3.5

In [ ]:
from scipy.interpolate import interp1d
from scipy.optimize import brentq

lam_points = np.array(sorted(gpt35_acc_by_lam.keys()))
acc_points = np.array([gpt35_acc_by_lam[l] for l in lam_points])

def estimate_lambda_crit(lam_arr, acc_arr, baseline):
    """Find λ where CoT accuracy crosses baseline"""
    f = interp1d(lam_arr, acc_arr - baseline, kind='linear', fill_value='extrapolate')
    try:
        for i in range(len(lam_arr) - 1):
            if (acc_arr[i] - baseline) * (acc_arr[i+1] - baseline) < 0:
                return brentq(f, lam_arr[i], lam_arr[i+1])
    except:
        pass
    # If no crossing, estimate from trend
    if acc_arr[-1] > baseline:
        return 1.0  # Never crossed
    return None

gpt35_lam_crit = estimate_lambda_crit(lam_points, acc_points, gpt35_direct_acc)

print('='*60)
print('λ* ESTIMATION FOR GPT-3.5')
print('='*60)

if gpt35_lam_crit is not None:
    print(f'\n  λ* = {gpt35_lam_crit:.3f}')
    print(f'  A* = {1 - gpt35_lam_crit:.3f}')
else:
    print('\n  Could not estimate λ* (no crossing detected)')
    print('  This may indicate GPT-3.5 is VERY robust to corruption')

## 12. A3 Scaling Law Visualization (3 Models)

In [ ]:
# Previous results (hardcoded)
models_data = {
    'GPT-3.5': {'baseline': gpt35_direct_acc * 100, 'lambda_crit': gpt35_lam_crit},
    'GPT-4o': {'baseline': 56.28, 'lambda_crit': 0.865},
    'Claude': {'baseline': 75.88, 'lambda_crit': 0.449}
}

# Filter valid data
valid_models = {k: v for k, v in models_data.items() if v['lambda_crit'] is not None}

fig, ax = plt.subplots(figsize=(12, 8))

# Plot points
colors = {'GPT-3.5': 'green', 'GPT-4o': 'blue', 'Claude': 'orange'}
for model, data in valid_models.items():
    ax.scatter(data['baseline'], data['lambda_crit'], 
               s=300, c=colors[model], label=model, zorder=5, edgecolors='black', linewidth=2)
    ax.annotate(f"{model}\n(B={data['baseline']:.1f}%, λ*={data['lambda_crit']:.3f})",
                (data['baseline'], data['lambda_crit']),
                textcoords="offset points", xytext=(15, 10),
                fontsize=11, ha='left')

# Fit line if 3+ points
if len(valid_models) >= 2:
    baselines = [v['baseline'] for v in valid_models.values()]
    lambdas = [v['lambda_crit'] for v in valid_models.values()]
    z = np.polyfit(baselines, lambdas, 1)
    p = np.poly1d(z)
    x_line = np.linspace(min(baselines) - 10, max(baselines) + 10, 100)
    ax.plot(x_line, p(x_line), 'r--', linewidth=2, alpha=0.7,
            label=f'Trend: λ* = {z[0]:.4f}×Baseline + {z[1]:.3f}')
    
    # Calculate correlation
    if len(valid_models) >= 3:
        corr = np.corrcoef(baselines, lambdas)[0, 1]
        ax.text(0.05, 0.05, f'Correlation: r = {corr:.3f}', 
                transform=ax.transAxes, fontsize=12,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Interpretation box
slope = z[0] if len(valid_models) >= 2 else 0
if slope < 0:
    interpretation = 'NEGATIVE CORRELATION\n\nHigher capability → Lower λ*\n→ "Smarter = More Gullible"\n\n✓ A3 (Nature) FEASIBLE'
    box_color = 'lightgreen'
elif slope > 0:
    interpretation = 'POSITIVE CORRELATION\n\nHigher capability → Higher λ*\n→ "Smarter = More Robust"\n\n✗ A3 requires different framing'
    box_color = 'lightyellow'
else:
    interpretation = 'NO CLEAR CORRELATION\n\nλ* appears independent of capability\n\n? More data needed'
    box_color = 'lightgray'

props = dict(boxstyle='round', facecolor=box_color, alpha=0.9)
ax.text(0.95, 0.95, interpretation, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', horizontalalignment='right', bbox=props)

ax.set_xlabel('Baseline Accuracy (%)', fontsize=14)
ax.set_ylabel('λ* (Backfire Boundary)', fontsize=14)
ax.set_title('A3 Scaling Law: Does Capability Predict Vulnerability?\n(Pilot with 3 Models)', fontsize=14)
ax.legend(loc='lower left', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(20, 90)
ax.set_ylim(0.2, 1.1)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR_EXP}/a3_scaling_law_3models.png', dpi=300, bbox_inches='tight')
plt.show()

print(f'\nSaved: {SAVE_DIR_EXP}/a3_scaling_law_3models.png')

## 13. Final Summary

In [ ]:
print('='*70)
print('A3 SCALING LAW PILOT: FINAL SUMMARY')
print('='*70)

print('''
┌─────────────────────────────────────────────────────────────────────┐
│                    THREE-MODEL COMPARISON                           │
├─────────────────────────────────────────────────────────────────────┤
│ Model       │ Baseline Accuracy │ λ* (Backfire)  │ Interpretation   │
├─────────────────────────────────────────────────────────────────────┤''')

for model, data in sorted(models_data.items(), key=lambda x: x[1]['baseline']):
    lam_str = f"{data['lambda_crit']:.3f}" if data['lambda_crit'] else "N/A"
    if data['baseline'] < 50:
        interp = "Low capability"
    elif data['baseline'] < 70:
        interp = "Medium capability"
    else:
        interp = "High capability"
    print(f"│ {model:<11} │ {data['baseline']:>15.1f}% │ {lam_str:>14} │ {interp:<16} │")

print('└─────────────────────────────────────────────────────────────────────┘')

# Calculate slope
if len(valid_models) >= 2:
    baselines = [v['baseline'] for v in valid_models.values()]
    lambdas = [v['lambda_crit'] for v in valid_models.values()]
    slope = np.polyfit(baselines, lambdas, 1)[0]
    
    print(f'\nTrend Analysis:')
    print(f'  Slope: {slope:.4f}')
    print(f'  For every 10% increase in baseline accuracy,')
    print(f'  λ* changes by {slope * 10:.3f}')
    
    if slope < -0.01:
        print(f'\n🎯 CONCLUSION: NEGATIVE CORRELATION CONFIRMED')
        print(f'   → Smarter models are MORE susceptible to trace contamination')
        print(f'   → A3 "Scaling Law of Vulnerability" is FEASIBLE for Nature')
    elif slope > 0.01:
        print(f'\n📊 CONCLUSION: POSITIVE CORRELATION')
        print(f'   → Smarter models are MORE robust')
        print(f'   → A3 needs reframing (not "vulnerability" story)')
    else:
        print(f'\n❓ CONCLUSION: WEAK/NO CORRELATION')
        print(f'   → More models needed to establish pattern')

## 14. Save Summary & Next Steps

In [ ]:
# Save summary JSON
summary = {
    'experiment': 'A3_Scaling_Law_Pilot',
    'date': EXPERIMENT_DATE,
    'n_problems_pilot': N_PROBLEMS_PILOT,
    'models': {
        'gpt35': {
            'model_name': API_MODEL,
            'baseline_accuracy': gpt35_direct_acc,
            'lambda_crit': gpt35_lam_crit,
            'accuracy_by_lambda': {str(k): v for k, v in gpt35_acc_by_lam.items()}
        },
        'gpt4o': {
            'model_name': 'gpt-4o',
            'baseline_accuracy': 0.5628,
            'lambda_crit': 0.865
        },
        'claude': {
            'model_name': 'claude-3-sonnet',
            'baseline_accuracy': 0.7588,
            'lambda_crit': 0.449
        }
    },
    'analysis': {
        'slope': float(slope) if 'slope' in dir() else None,
        'correlation_direction': 'negative' if slope < 0 else 'positive' if slope > 0 else 'none',
        'a3_feasibility': 'high' if slope < -0.01 else 'low'
    },
    'next_steps': [
        'If negative correlation confirmed: Proceed with full A3 experiment (200 problems, 5+ models)',
        'Add more models: Llama-7B, Llama-70B, Mistral-7B, etc.',
        'Consider different capability metrics (MMLU scores, parameter count)'
    ]
}

save_json(summary, f'{SAVE_DIR_EXP}/results/a3_pilot_summary.json')

print('\n' + '='*70)
print('EXPERIMENT COMPLETE')
print('='*70)
print(f'\nAll results saved to: {SAVE_DIR_EXP}')
print('\nNext Steps:')
for i, step in enumerate(summary['next_steps'], 1):
    print(f'  {i}. {step}')